In [1]:
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np

def knn_impute(df, n_neighbors=5):
    non_empty_columns = df.columns[df.isnull().mean() < 1.0]
    df_reduced = df[non_empty_columns].copy()

    for col in df_reduced.select_dtypes(include='object').columns:
        df_reduced[col] = df_reduced[col].astype('category').cat.codes

    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_reduced), columns=df_reduced.columns)
    
    for col in df.select_dtypes(include='object').columns:
        if col in df_reduced.columns:
            df_imputed[col] = df_imputed[col].round().astype(int).map(
                dict(enumerate(df[col].astype('category').cat.categories))
            )
    
    for col in df.columns:
        if col not in df_imputed.columns:
            df_imputed[col] = np.nan
    
    df_imputed = df_imputed[df.columns]
    
    return df_imputed

In [2]:
df = pd.read_csv('processed_train_data.csv')

df.drop(columns=['Dogum Yili', 'Dogum Tarihi',"Hangi STK'nin Uyesisiniz?", "Uye Oldugunuz Kulubun Ismi",
    "Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?","Spor Dalindaki Rolunuz Nedir?",'Lise Adi Diger',
    'Lise Bolum Diger','Lise Adi', 'Lise Bolumu', 'Lise Sehir','Baska Kurumdan Aldigi Burs Miktari'], inplace=True)

df_train_imputed = knn_impute(df, n_neighbors=5)
df_train_imputed.to_csv('train_imputed.csv', index=False)

C:\Users\ferha\AppData\Local\Temp\ipykernel_9920\2044820398.py:1: DtypeWarning: Columns (13,15,19,32,37,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_train_data.csv')
